<a href="https://colab.research.google.com/github/bluetinue/Country_Name/blob/main/%E5%9F%BA%E4%BA%8EGRU%E7%9A%84seq2seq%E7%9A%84%E8%8B%B1%E8%AF%91%E6%B3%95%E6%A1%88%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title 导包
from google.colab import drive
drive.mount('/content/drive')

# 用于正则表达式
import re
# 用于构建网络结构和函数的torch工具包
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# torch中预定义的优化方法工具包
import torch.optim as optim
import time
# 用于随机生成数据
import random
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#@title 全局变量

#开始字符标注
SOS_TOKEN = 0
EOS_TOKEN = 1

#最大句子长度
MAX_LENGTH = 10

data_path = "/content/drive/MyDrive/NLP/data/eng-fra-v2.txt"

In [29]:
#@title 文本清洗工具函数
def normal2String(s):
  s1 = s.lower().strip()
  s2 = re.sub(r"([.!?])", r" \1 ", s1)
  s3 = re.sub(r"[^a-zA-Z.!?]+", r" ", s2)
  return s3.strip()

In [45]:
#@title 对原始数据进行预处理
#构建出[[英文,法文，.....的列表对]]
def wash_data():
  with open(data_path,"r",encoding="utf-8") as fr:
    lines = fr.readlines()
  #列表推导式
  my_pairs = [[ normal2String(s) for s in i.strip().split("\t")] for i in lines]

  #初始化词表，默认有开始和结束分隔符和初始长度
  english_word2index = {"SOS":0,"EOS":1}
  english_word2index_n = 2
  fres_word2index = {"SOS":0,"EOS":1}
  fres_word2index_n = 2

  #构造两个词表的word2index表
  for pair in my_pairs:
    for word in pair[0].split(" "):
      if word not in english_word2index:
        english_word2index[word] = len(english_word2index)
        #english_word2index[word] = english_word2index_n
        #english_word2index_n += 1

    for word in pair[1].split(" "):
      if word not in fres_word2index:
        fres_word2index[word] = len(fres_word2index)
        #fres_word2index[word] = fres_word2index_n
        #fres_word2index_n += 1

  #构造两个词表的index2word表
  english_index2word = {v:k for k,v in english_word2index.items()}
  fres_index2word = {v:k for k,v in fres_word2index.items()}
  return english_word2index,english_index2word,\
    len(english_word2index),fres_word2index,fres_index2word,\
    len(fres_index2word),my_pairs

In [47]:
data = wash_data()

In [51]:
print("english_number-->",data[2])
print("english_word-->",data[0])
print("fresh_number-->",data[5])
print("fresh_word-->",data[3])

english_number--> 2803
english_word--> {'SOS': 0, 'EOS': 1, 'i': 2, 'm': 3, '.': 4, 'ok': 5, 'fat': 6, 'fit': 7, 'hit': 8, '!': 9, 'ill': 10, 'sad': 11, 'shy': 12, 'wet': 13, 'he': 14, 's': 15, 'am': 16, 'back': 17, 'bald': 18, 'busy': 19, 'calm': 20, 'cold': 21, 'done': 22, 'fine': 23, 'free': 24, 'full': 25, 'glad': 26, 'home': 27, 'late': 28, 'lazy': 29, 'okay': 30, 'safe': 31, 'sick': 32, 'sure': 33, 'tall': 34, 'thin': 35, 'tidy': 36, 'ugly': 37, 'weak': 38, 'well': 39, 'is': 40, 'old': 41, 'a': 42, 'dj': 43, 'good': 44, 'rich': 45, 'here': 46, 'cop': 47, 'man': 48, 'alone': 49, 'armed': 50, 'awake': 51, 'blind': 52, 'broke': 53, 'crazy': 54, 'cured': 55, 'drunk': 56, 'dying': 57, 'early': 58, 'first': 59, 'fussy': 60, 'going': 61, 'loyal': 62, 'lucky': 63, 'lying': 64, 'quiet': 65, 'ready': 66, 'right': 67, 'sober': 68, 'sorry': 69, 'stuck': 70, 'timid': 71, 'tired': 72, 'tough': 73, 'yours': 74, 'she': 75, 'hot': 76, 'we': 77, 're': 78, 'kind': 79, 'poor': 80, 'swiss': 81, 'smar

In [9]:
#@title 构建基于GRU的编码器

In [ ]:
#@title 构建基于GRU的解码器

In [ ]:
#@title 构建模型评估函数

In [ ]:
#@title 构建模型测试函数